In [0]:
#SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
spark = SparkSession.builder.appName("UseCase").getOrCreate()


In [0]:

def make_connection(storage_account_name,storage_account_access_key):
    try:
        spark.conf.set(
        "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
        storage_account_access_key)
        return True
    except Exception as e:
        raise Exception("Not able to make connection with storage account")
        return False


In [0]:
def read_file(file_location):
    df = spark.read.format("json").option("inferSchema", "true").load(file_location)
    return df
    

In [0]:
def add_cols(df):
    df1=df.withColumn("timestamp",current_timestamp())
    return df1


In [0]:

storage_account_access_key = dbutils.secrets.get('usecaseScope','storage_account_access_key')
storage_account_name = dbutils.secrets.get('usecaseScope','storage_account_name')
container_name =  dbutils.secrets.get('usecaseScope','container_name')
#file_location = "wasbs://"+container_name+"@"+storage_account_name+".blob.core.windows.net/json_data/movies.json"
fn=dbutils.widgets.get("file_name")
file_location = "wasbs://"+container_name+"@"+storage_account_name+".blob.core.windows.net/json_data/"+fn

##Call the defined functions
try:
    connection=make_connection(storage_account_name,storage_account_access_key)
    print(f"connection , {connection}")
    df=read_file(file_location)
    df1=add_cols(df)
    #df1.show(5)

    ##Write df into delta table i.e ingest_table
    df1.write.partitionBy("timestamp")\
                .mode("append")\
                .format("delta")\
                .option("overwriteSchema", "true")\
                .save("/delta/ingest_table")
    print(True)

except Exception as e:
    raise Exception(e)
